#Text Summarization
The idea of document summarization is a
bit different from keyphrase extraction or topic modeling. In this case, the end result
is still in the form of some document, but with a few sentences based on the length we
might want the summary to be. This is similar to an abstract or an executive summary
in a research paper. The main objective of automated document summarization is
to perform this summarization without involving human input, except for running
computer programs. Mathematical and statistical models help in building and
automating the task of summarizing documents by observing their content and context.

There are two broad approaches to document summarization using automated
techniques. They are described as follows:
- __Extraction-based techniques:__ These methods use mathematical
and statistical concepts like SVD to extract some key subset of the
content from the original document such that this subset of content
contains the core information and acts as the focal point of the entire
document. This content can be words, phrases, or even sentences.
The end result from this approach is a short executive summary of a
couple of lines extracted from the original document. No new content
is generated in this technique, hence the name extraction-based.
- __Abstraction-based techniques:__ These methods are more complex
and sophisticated. They leverage language semantics to create
representations and use natural language generation (NLG)
techniques where the machine uses knowledge bases and semantic
representations to generate text on its own and create summaries
just like a human would write them. Thanks to deep learning, we can
implement these techniques easily but they require a lot of data and
compute.

We will cover extraction based methods here due to constraints of needed a lot of data + compute for abstraction based methods. But you can leverage the seq2seq models you learnt in language translation on an appropriate dataset to build deep learning based abstractive summarizers


#Extraction based techniques


## Install necessary dependencies

In [1]:
import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Get Text Document

We use the description of a very popular role-playing game (RPG) Skyrim from
Bethesda Softworks for summarization. 

In [2]:
DOCUMENT = """
The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios and published by Bethesda Softworks.

It is the fifth main installment in The Elder Scrolls series, following The Elder Scrolls IV: Oblivion.
"""

In [3]:
import re
DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT) #Combining all the paragraphs
DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
DOCUMENT = DOCUMENT.strip()

In [4]:
print(DOCUMENT)

The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios and published by Bethesda Softworks. It is the fifth main installment in The Elder Scrolls series, following The Elder Scrolls IV: Oblivion.


Sentences Collection

In [5]:
sentences = nltk.sent_tokenize(DOCUMENT)
print(sentences)
print("No. of sentences:",len(sentences))

['The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios and published by Bethesda Softworks.', 'It is the fifth main installment in The Elder Scrolls series, following The Elder Scrolls IV: Oblivion.']
No. of sentences: 2


## Basic Text pre-processing

In [6]:
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document) #Function Def Vectorize
norm_sentences = normalize_corpus(sentences)
norm_sentences[:3]

array(['elder scrolls v skyrim action roleplaying video game developed bethesda game studios published bethesda softworks',
       'fifth main installment elder scrolls series following elder scrolls iv oblivion'],
      dtype='<U113')

## *I - UNIGRAM*

In [7]:
# Creation of Unigarms
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range =(1, 1))
X1 = vectorizer.fit_transform(norm_sentences)
unigrams = (vectorizer.get_feature_names())
print(unigrams)
print(len(unigrams))

['action', 'bethesda', 'developed', 'elder', 'fifth', 'following', 'game', 'installment', 'iv', 'main', 'oblivion', 'published', 'roleplaying', 'scrolls', 'series', 'skyrim', 'softworks', 'studios', 'video']
19


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


###TF

In [8]:
print("Norm Sentences:", norm_sentences)
print("Len of Norm Sentences:", len(norm_sentences))

Norm Sentences: ['elder scrolls v skyrim action roleplaying video game developed bethesda game studios published bethesda softworks'
 'fifth main installment elder scrolls series following elder scrolls iv oblivion']
Len of Norm Sentences: 2


In [9]:
def computeTF(doc):
  valTF = []
  for each in doc:
    wordDict = dict.fromkeys(unigrams, 0)
    sentence = each.split(" ")

    for word in sentence:
      if len(word) == 1:
        sentence.remove(word)

    for word in sentence:
      wordDict[word]+=1

    res = []
    for i in wordDict:
      comp = float(wordDict[i] / len(each))
      res.append(round(comp, 4))
    
    valTF.append(res)
  return(valTF)
    
TF = computeTF(norm_sentences)

In [10]:
TF = np.array(TF)
TF = TF.T
type(TF)

numpy.ndarray

In [11]:
df = pd.DataFrame(TF, index=unigrams)
df.sort_index(ascending=True).head(10)

,0,1
action,0.0088,0.0000
bethesda,0.0177,0.0000
developed,0.0088,0.0000
elder,0.0088,0.0253
fifth,0.0000,0.0127
following,0.0000,0.0127
game,0.0177,0.0000
installment,0.0000,0.0127
iv,0.0000,0.0127
main,0.0000,0.0127


###DF (Document Frequency/Term Presence)

In [12]:
DF = vectorizer.fit_transform(norm_sentences)
DF = DF.toarray()
DF = DF.T
DF

array([[1, 0],
       [2, 0],
       [1, 0],
       [1, 2],
       [0, 1],
       [0, 1],
       [2, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 2],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

In [13]:
df = pd.DataFrame(DF, index=unigrams)
df

,0,1
action,1,0
bethesda,2,0
developed,1,0
elder,1,2
fifth,0,1
following,0,1
game,2,0
installment,0,1
iv,0,1
main,0,1


###TF-IDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
dt_matrix = tv.fit_transform(norm_sentences)
dt_matrix = dt_matrix.toarray()
dt_matrix

array([[0.24244659, 0.48489319, 0.24244659, 0.17250275, 0.        ,
        0.        , 0.48489319, 0.        , 0.        , 0.        ,
        0.        , 0.24244659, 0.24244659, 0.17250275, 0.        ,
        0.24244659, 0.24244659, 0.24244659, 0.24244659],
       [0.        , 0.        , 0.        , 0.42808473, 0.30082908,
        0.30082908, 0.        , 0.30082908, 0.30082908, 0.30082908,
        0.30082908, 0.        , 0.        , 0.42808473, 0.30082908,
        0.        , 0.        , 0.        , 0.        ]])

In [15]:
td_matrix = dt_matrix.T #Transpose Matrix
td_matrix
print(td_matrix.shape)

(19, 2)


In [16]:
pd.DataFrame(np.round(td_matrix, 2), index=unigrams)

,0,1
action,0.24,0.00
bethesda,0.48,0.00
developed,0.24,0.00
elder,0.17,0.43
fifth,0.00,0.30
following,0.00,0.30
game,0.48,0.00
installment,0.00,0.30
iv,0.00,0.30
main,0.00,0.30


## *II - BIGRAM* 

In [17]:
# Forming Bigrams
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (2,2))
bigrams_matrix = vectorizer.fit_transform(norm_sentences) 
bigrams = (vectorizer.get_feature_names())
print(bigrams)

['action roleplaying', 'bethesda game', 'bethesda softworks', 'developed bethesda', 'elder scrolls', 'fifth main', 'following elder', 'game developed', 'game studios', 'installment elder', 'iv oblivion', 'main installment', 'published bethesda', 'roleplaying video', 'scrolls iv', 'scrolls series', 'scrolls skyrim', 'series following', 'skyrim action', 'studios published', 'video game']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### TF-IDF

In [18]:
# Applying TFIDF
vectorizer = TfidfVectorizer(ngram_range = (2, 2))
bigram_matrix = vectorizer.fit_transform(norm_sentences)
bigram_matrix = bigram_matrix.toarray()
bigram_matrix = bigram_matrix.T
print("\n\nScores : \n", bigram_matrix)



Scores : 
 [[0.2827721  0.        ]
 [0.2827721  0.        ]
 [0.2827721  0.        ]
 [0.2827721  0.        ]
 [0.20119468 0.44943642]
 [0.         0.3158336 ]
 [0.         0.3158336 ]
 [0.2827721  0.        ]
 [0.2827721  0.        ]
 [0.         0.3158336 ]
 [0.         0.3158336 ]
 [0.         0.3158336 ]
 [0.2827721  0.        ]
 [0.2827721  0.        ]
 [0.         0.3158336 ]
 [0.         0.3158336 ]
 [0.2827721  0.        ]
 [0.         0.3158336 ]
 [0.2827721  0.        ]
 [0.2827721  0.        ]
 [0.2827721  0.        ]]


In [19]:
pd.DataFrame(np.round(bigram_matrix, 2), index=bigrams)

,0,1
action roleplaying,0.28,0.00
bethesda game,0.28,0.00
bethesda softworks,0.28,0.00
developed bethesda,0.28,0.00
elder scrolls,0.20,0.45
fifth main,0.00,0.32
following elder,0.00,0.32
game developed,0.28,0.00
game studios,0.28,0.00
installment elder,0.00,0.32


# *III - Trigrams*

In [20]:
# Forming Trigrams
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range = (3,3))
trigrams_matrix = vectorizer.fit_transform(norm_sentences) 
trigrams = (vectorizer.get_feature_names())
print(trigrams)

['action roleplaying video', 'bethesda game studios', 'developed bethesda game', 'elder scrolls iv', 'elder scrolls series', 'elder scrolls skyrim', 'fifth main installment', 'following elder scrolls', 'game developed bethesda', 'game studios published', 'installment elder scrolls', 'main installment elder', 'published bethesda softworks', 'roleplaying video game', 'scrolls iv oblivion', 'scrolls series following', 'scrolls skyrim action', 'series following elder', 'skyrim action roleplaying', 'studios published bethesda', 'video game developed']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## TF-IDF

In [21]:
# Applying TFIDF
vectorizer = TfidfVectorizer(ngram_range = (3, 3))
trigrams_matrix = vectorizer.fit_transform(norm_sentences)
trigrams_matrix = trigrams_matrix.toarray()
trigrams_matrix = trigrams_matrix.T
print("\n\nScores : \n", trigrams_matrix)



Scores : 
 [[0.28867513 0.        ]
 [0.28867513 0.        ]
 [0.28867513 0.        ]
 [0.         0.33333333]
 [0.         0.33333333]
 [0.28867513 0.        ]
 [0.         0.33333333]
 [0.         0.33333333]
 [0.28867513 0.        ]
 [0.28867513 0.        ]
 [0.         0.33333333]
 [0.         0.33333333]
 [0.28867513 0.        ]
 [0.28867513 0.        ]
 [0.         0.33333333]
 [0.         0.33333333]
 [0.28867513 0.        ]
 [0.         0.33333333]
 [0.28867513 0.        ]
 [0.28867513 0.        ]
 [0.28867513 0.        ]]


In [22]:
pd.DataFrame(np.round(bigram_matrix, 2), index=bigrams)

,0,1
action roleplaying,0.28,0.00
bethesda game,0.28,0.00
bethesda softworks,0.28,0.00
developed bethesda,0.28,0.00
elder scrolls,0.20,0.45
fifth main,0.00,0.32
following elder,0.00,0.32
game developed,0.28,0.00
game studios,0.28,0.00
installment elder,0.00,0.32
